In [1]:
import sys
import os

# Add the project root directory to sys.path
project_root = os.path.abspath(".") 
if project_root not in sys.path:
    sys.path.append(project_root)
print(sys.path[-1])


from reward_func.evo_devo import coord_reward_func, oscillator_reward_func, somitogenesis_reward_func


/Users/dannyhuang/Developer/gflownet2/base


In [2]:
import time

test_state = (50, -53, -57, 8, 9, -6, -117, 81, 8)

start_time = time.perf_counter_ns()
test_reward = coord_reward_func(test_state)
end_time = time.perf_counter_ns()
print(f"Time taken to run coord_reward_func: {(end_time - start_time)/1e9:.9f} seconds")

print(f"Test reward for state {test_state}: {test_reward}")
    



# test_weights = [-5, 2, 3, 1, 2, -1, 2, 1, -3] 
# test_weights = [0, -10, 0, 4, 6, -10, -10, 0, 0]  # w12=-5, w22=..., w23=-10, w31=-10 
# test_weights = [154, -200, 82, -33, 90, -90, -82, -13, 30] 
# reward = oscillator_reward_func(test_weights, plot=True)
# print(f"Reward for oscillator: {reward}")

Time taken to run coord_reward_func: 0.000024791 seconds
Test reward for state (50, -53, -57, 8, 9, -6, -117, 81, 8): 4


In [4]:
# Import plotting function from graph module
from graph.graph import draw_network_motif
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
import numpy as np



test_weights = [80, -35, -155, 186, -20, -20]
# test_weights = [0, 0, 0, 0, 0, 0]
n_nodes = int((-1 + (1 + 4*len(test_weights))**0.5) / 2)  # solve quadratic: n^2 + n - len(state) = 0
n_weights = n_nodes * n_nodes

def update_plot(**kwargs):
    params = list(kwargs.values())
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(7, 12))
    # n_weights = n_nodes * n_nodes
    # weights = params[:n_weights]
    draw_network_motif(params, ax=ax2)
    ax2.set_title(f"{n_nodes}-Node Network Motif")
    
    # Plot somite pattern and get reward
    start_time = time.perf_counter_ns()
    reward = somitogenesis_reward_func(params, plot=True, ax=ax1)
    print(f"Reward for somitogenesis: {reward}")
    end_time = time.perf_counter_ns()
    print(f"Time taken to run somitogenesis_reward_func: {(end_time - start_time)/1e9:.9f} seconds")
    
    plt.tight_layout()

# Create sliders for all parameters
sliders = {}
# Weight sliders
for i in range(n_weights):
    default_value = test_weights[i] if i < len(test_weights) else 0
    sliders[f'w{i+1}'] = FloatSlider(min=-100, max=100, step=1, value=default_value)
# D value sliders    
for i in range(n_nodes):
    default_value = test_weights[n_weights+i] if n_weights+i < len(test_weights) else 0
    sliders[f'd{i+1}'] = FloatSlider(min=-10, max=10, step=1, value=default_value)

interact(update_plot, **sliders)


interactive(children=(FloatSlider(value=80.0, description='w1', min=-100.0, step=1.0), FloatSlider(value=-35.0…

<function __main__.update_plot(**kwargs)>

In [10]:
# # Import plotting function from graph module
# from graph.graph import plot_network_motifs_and_somites

# # Define test weights to visualize
# test_weights_list = [
#     # (65, -110, 52, -40, 32, -8, -65, -32, 71),
#     [-5, 200, -6, -51, -26, 5, 5, 1, -125, 25, 30, 100, 0, 60, -5, -25, -1, 5, 0, -75, 30, 0, -1, -200, -25],
#     [60, 32, -38, -85, 70, -63, 22, -27, -7],
#     [60, 32, -38, -85, 70, -63, 22, -27, -7, 0, 0, 0, 0, 0, 0, 0]
# ]

# # Plot network motifs and their corresponding somite patterns
# save_path = plot_network_motifs_and_somites(test_weights_list)
# print(f"Plot saved to: {save_path}")



In [3]:
# Test cases for Progressive Masking in GridEnv2
import numpy as np
from types import SimpleNamespace
from disc_gflownet.envs.grid_env2 import GridEnv2

# Create a simple test environment with 3 nodes (12 dimensions: 9 weights + 3 diagonals)
args = SimpleNamespace(
    n_workers=1,
    cache_max_size=1000,
    min_reward=0.001,
    custom_reward_fn=lambda x: 0,  # Dummy reward function
    actions_per_dim={'weight': [1, 5, 25, -1, -5, -25], 'diagonal': [1, 5, -1, -5]},
    grid_bound={'weight': {'min': -200, 'max': 200}, 'diagonal': {'min': -20, 'max': 20}},
    enable_time=False,
    consistent_signs=True,
    n_dims=3**2+3,  # 9 weights + 3 diagonals
    n_steps=2+6+10,  # Total steps for all network sizes
    steps_per_network={1:2, 2:6, 3:10}  # Steps per network size
)

env = GridEnv2(args)

# Print all actions first
env.print_actions()

print("Testing Progressive Masking in GridEnv2")
print(f"Environment has {env.n_nodes} nodes, {env.n_dims} dimensions")
print(f"Action space size: {env.action_dim}")


# s0
env.reset()
print(f"Current network size: {env.current_network_size}") 
print(f"Steps Total: {env._step}")
print(f"_step in current network: {env._step_in_current_network}")
mask = env.get_forward_mask(env._state)
print(f"Number of allowed actions: {np.sum(mask)}")
allowed_indices = np.where(mask)[0]
print(f"Allowed action indices: {allowed_indices}")
print(f"--------\n") 

# First action
action = allowed_indices[0]
next_state, reward, done = env.step(action)
print("\nTaking action:", action)
print(f"New state: {env._state}")
print(f"Done: {done}")

# s1
print(f"Current network size: {env.current_network_size}") 
print(f"Steps Total: {env._step}")
print(f"_step in current network: {env._step_in_current_network}")
mask = env.get_forward_mask(env._state)
print(f"Number of allowed actions: {np.sum(mask)}")
allowed_indices = np.where(mask)[0]
print(f"Allowed action indices: {allowed_indices}")
print(f"--------\n") 

# Second action
action = allowed_indices[0]
next_state, reward, done = env.step(action)
print("\nTaking action:", action)
print(f"New state: {env._state}")
print(f"Done: {done}")

# s2
print(f"Current network size: {env.current_network_size}") 
print(f"Steps Total: {env._step}")
print(f"_step in current network: {env._step_in_current_network}")
mask = env.get_forward_mask(env._state)
print(f"Number of allowed actions: {np.sum(mask)}")
allowed_indices = np.where(mask)[0]
print(f"Allowed action indices: {allowed_indices}")
print(f"--------\n") 

# Third action
action = allowed_indices[0]
next_state, reward, done = env.step(action)
print("\nTaking action:", action)
print(f"New state: {env._state}")
print(f"Done: {done}")

# s3
print(f"Current network size: {env.current_network_size}") 
print(f"Steps Total: {env._step}")
print(f"_step in current network: {env._step_in_current_network}")
mask = env.get_forward_mask(env._state)
print(f"Number of allowed actions: {np.sum(mask)}")
allowed_indices = np.where(mask)[0]
print(f"Allowed action indices: {allowed_indices}")
print(f"--------\n") 

# Fourth action
action = allowed_indices[0]
next_state, reward, done = env.step(action)
print("\nTaking action:", action)
print(f"New state: {env._state}")
print(f"Done: {done}")

# s4
print(f"Current network size: {env.current_network_size}") 
print(f"Steps Total: {env._step}")
print(f"_step in current network: {env._step_in_current_network}")
mask = env.get_forward_mask(env._state)
print(f"Number of allowed actions: {np.sum(mask)}")
allowed_indices = np.where(mask)[0]
print(f"Allowed action indices: {allowed_indices}")
print(f"--------\n") 

# Fifth action
action = allowed_indices[0]
next_state, reward, done = env.step(action)
print("\nTaking action:", action)
print(f"New state: {env._state}")
print(f"Done: {done}")

# s5
print(f"Current network size: {env.current_network_size}") 
print(f"Steps Total: {env._step}")
print(f"_step in current network: {env._step_in_current_network}")
mask = env.get_forward_mask(env._state)
print(f"Number of allowed actions: {np.sum(mask)}")
allowed_indices = np.where(mask)[0]
print(f"Allowed action indices: {allowed_indices}")
print(f"--------\n") 

# Sixth action
action = allowed_indices[0]
next_state, reward, done = env.step(action)
print("\nTaking action:", action)
print(f"New state: {env._state}")
print(f"Done: {done}")

# s6
print(f"Current network size: {env.current_network_size}") 
print(f"Steps Total: {env._step}")
print(f"_step in current network: {env._step_in_current_network}")
mask = env.get_forward_mask(env._state)
print(f"Number of allowed actions: {np.sum(mask)}")
allowed_indices = np.where(mask)[0]
print(f"Allowed action indices: {allowed_indices}")
print(f"--------\n") 

# Seventh action
action = allowed_indices[0]
next_state, reward, done = env.step(action)
print("\nTaking action:", action)
print(f"New state: {env._state}")
print(f"Done: {done}")

# s7
print(f"Current network size: {env.current_network_size}") 
print(f"Steps Total: {env._step}")
print(f"_step in current network: {env._step_in_current_network}")
mask = env.get_forward_mask(env._state)
print(f"Number of allowed actions: {np.sum(mask)}")
allowed_indices = np.where(mask)[0]
print(f"Allowed action indices: {allowed_indices}")
print(f"--------\n") 

# Eighth action
action = allowed_indices[0]
next_state, reward, done = env.step(action)
print("\nTaking action:", action)
print(f"New state: {env._state}")
print(f"Done: {done}")

# s8
print(f"Current network size: {env.current_network_size}") 
print(f"Steps Total: {env._step}")
print(f"_step in current network: {env._step_in_current_network}")
mask = env.get_forward_mask(env._state)
print(f"Number of allowed actions: {np.sum(mask)}")
allowed_indices = np.where(mask)[0]
print(f"Allowed action indices: {allowed_indices}")
print(f"--------\n") 

# Ninth action
action = allowed_indices[0]
next_state, reward, done = env.step(action)
print("\nTaking action:", action)
print(f"New state: {env._state}")
print(f"Done: {done}")

# s9
print(f"Current network size: {env.current_network_size}") 
print(f"Steps Total: {env._step}")
print(f"_step in current network: {env._step_in_current_network}")
mask = env.get_forward_mask(env._state)
print(f"Number of allowed actions: {np.sum(mask)}")
allowed_indices = np.where(mask)[0]
print(f"Allowed action indices: {allowed_indices}")
print(f"--------\n") 

# Tenth action
action = allowed_indices[0]
next_state, reward, done = env.step(action)
print("\nTaking action:", action)
print(f"New state: {env._state}")
print(f"Done: {done}")

# s10
print(f"Current network size: {env.current_network_size}") 
print(f"Steps Total: {env._step}")
print(f"_step in current network: {env._step_in_current_network}")
mask = env.get_forward_mask(env._state)
print(f"Number of allowed actions: {np.sum(mask)}")
allowed_indices = np.where(mask)[0]
print(f"Allowed action indices: {allowed_indices}")
print(f"--------\n") 


------------------------------------------
All available actions (action_names): 66
['(0): weight0 1', '(1): weight0 5', '(2): weight0 25', '(3): weight0 -1', '(4): weight0 -5', '(5): weight0 -25', '(6): weight1 1', '(7): weight1 5', '(8): weight1 25', '(9): weight1 -1', '(10): weight1 -5', '(11): weight1 -25', '(12): weight2 1', '(13): weight2 5', '(14): weight2 25', '(15): weight2 -1', '(16): weight2 -5', '(17): weight2 -25', '(18): weight3 1', '(19): weight3 5', '(20): weight3 25', '(21): weight3 -1', '(22): weight3 -5', '(23): weight3 -25', '(24): weight4 1', '(25): weight4 5', '(26): weight4 25', '(27): weight4 -1', '(28): weight4 -5', '(29): weight4 -25', '(30): weight5 1', '(31): weight5 5', '(32): weight5 25', '(33): weight5 -1', '(34): weight5 -5', '(35): weight5 -25', '(36): weight6 1', '(37): weight6 5', '(38): weight6 25', '(39): weight6 -1', '(40): weight6 -5', '(41): weight6 -25', '(42): weight7 1', '(43): weight7 5', '(44): weight7 25', '(45): weight7 -1', '(46): weight7

In [ ]:
import numpy as np
def sparsity_reward_combined(state, w1=0.0, w2=1.0):
    # Entropy-based component
    # Normalize values to probabilities
    abs_values = np.abs(state)
    if sum(abs_values) == 0:
        entropy_reward = 1.0  # maximum sparsity
    else:
        probs = abs_values / sum(abs_values)
        # Calculate entropy (lower entropy = more sparse)
        entropy = -sum(p * np.log(p) for p in probs if p > 0)
        entropy_reward = 1 / (1 + entropy)  # transform to reward
    
    # L0 component (explicitly rewards zeros)
    n_zeros = sum(1 for x in state if x == 0)
    l0_reward = n_zeros / len(state)
    
    return w1 * entropy_reward + w2 * l0_reward

# Example states to test
sparse_state1 = (10, 0, 0, 0, 10, 0, 0, 0, 10)
sparse_state2 = (0, 0, 0, 0, 0, 0, 0, 0, 0)

# Test combined reward function
print(f"Combined sparsity reward for sparse_state1: {sparsity_reward_combined(sparse_state1):.4f}")
print(f"Combined sparsity reward for sparse_state2: {sparsity_reward_combined(sparse_state2):.4f}")